# CNN Model for MNIST Detection

In [ ]:
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import math

# Enable inline plotting
%matplotlib inline

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
''' Build a simple MNIST classification CNN
    The network takes ~3 minutes to train on a normal laptop and reaches roughly 97% of accuracy
    Model structure: Conv, Conv, Max pooling, Dropout, Dense, Dense
'''
def build_mnist_model():
    
    activation = 'relu'
    # input image dimensions
    img_rows, img_cols, img_colors = 28, 28, 1
    
    model = keras.Sequential()
    model.add(layers.Conv2D(8, kernel_size=(3, 3), input_shape=(img_rows, img_cols, img_colors), activation=activation))
    model.add(layers.Conv2D(8, (3, 3), activation=activation))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Dropout(0.25))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation=activation))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(num_classes))
    model.add(layers.Activation('softmax', name='y_pred'))
              
    return model

In [ ]:
''' Normalize input to the range of [0..1]
    Apart from assisting in the convergance of the training process, this 
    will also make our lives easier during the adversarial attack process
'''
def normalize(x_train,x_test):
    x_train -= x_train.min()
    x_train /= x_train.max()
    x_test -= x_test.min()
    x_test /= x_test.max()
    
    return x_train, x_test

In [ ]:
# Load and prepare the datasets for training
num_classes = 10

img_rows, img_cols, img_colors = 28, 28, 1
(train_images, train_labels), (test_images, test_labels) = keras.datasets.mnist.load_data()
train_images = train_images.astype('float32')
test_images = test_images.astype('float32')
train_images = train_images.reshape(train_images.shape[0], img_rows, img_cols, 1)
test_images = test_images.reshape(test_images.shape[0], img_rows, img_cols, 1)
train_images, test_images = normalize(train_images, test_images)
sanity_images = test_images[:10]
    
train_labels = keras.utils.to_categorical(train_labels, num_classes)
test_labels = keras.utils.to_categorical(test_labels, num_classes)
sanity_labels = test_labels[:10]

In [ ]:
# Train the model
batch_size = 128
maxepoches = 12
learning_rate = 0.1
lr_decay = 1e-6
lr_drop = 20

sess = tf.Session()
keras.backend.set_session(sess)

def lr_scheduler(epoch):
    return learning_rate * (0.5 ** (epoch // lr_drop))
reduce_lr = keras.callbacks.LearningRateScheduler(lr_scheduler)

model = build_mnist_model()

model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.Adadelta(),
              metrics=[keras.metrics.CategoricalAccuracy()])

history = model.fit(train_images, train_labels,
                    batch_size=batch_size,
                    epochs=maxepoches,
                    verbose=1,
                    validation_data=(test_images, test_labels),
                    callbacks=[reduce_lr])


# Attack Utilities

In [ ]:
''' A simple utility funcion for evaluating the success of an attack
'''
def TestAttack(model, adv_images, orig_images, true_labels, target_labels=None, targeted=False):
    score = model.evaluate(adv_images, true_labels, verbose=0)
    print('Test loss: {:.2f}'.format(score[0]))
    print('Successfully moved out of source class: {:.2f}'.format( 1 - score[1]))
    
    if targeted:
        score = model.evaluate(adv_images, target_labels, verbose=0)
        print('Test loss: {:.2f}'.format(score[0]))
        print('Successfully perturbed to target class: {:.2f}'.format(score[1]))
    
    dist = np.mean(np.sqrt(np.mean(np.square(adv_images - orig_images), axis=(1,2,3))))
    print('Mean perturbation distance: {:.2f}'.format(dist))
    
    index = 3
    img = adv_images[index].reshape(28, 28)
    plt.imshow(img, cmap='gray')
    plt.show()
    
    ret_score = score[1] if targeted else 1 - score[1]
    return ret_score, dist

# Part I - TF Implementation for PGD Attack

## 1. FGSM and TGSM implementations against the MNIST testing set

In [ ]:
''' Fast Gradient Sign Method implementation - perturb all input features by an epsilon sized step in 
    the direction of loss gradient
'''
def FGSM(model, images, labels, epsilon=0.3):
    # define fgsm model session
    adv_loss = keras.losses.categorical_crossentropy(labels, model.output)
    grads_wrt_input = keras.backend.gradients(adv_loss, model.input)[0]
    sign_grads = keras.backend.sign(grads_wrt_input)
    delta = epsilon * sign_grads
    images_adv  = images + delta

    # run session
    sess = keras.backend.get_session()
    adv_out = sess.run(images_adv, feed_dict={model.input: images})
    
    return adv_out

In [ ]:
images_adv = FGSM(model, test_images, test_labels)

In [ ]:
# Compute perturbations using FGSM
adv_images = FGSM(model, test_images, test_labels, epsilon=0.3)
TestAttack(model, adv_images, test_images, test_labels, targeted=False)

In [ ]:
''' Targeted Gradient Sign Method implementation - A targeted variant of the FGSM attack
    here we minimize the loss with respect to the target class, as opposed to maximizing the loss with respect
    to the source class
'''
def TGSM(model, images, target, epsilon=0.3):
    # define fgsm model session
    adv_loss = keras.losses.categorical_crossentropy(target, model.output)
    grads_wrt_input = keras.backend.gradients(adv_loss, model.input)[0]
    sign_grads = keras.backend.sign(grads_wrt_input)
    delta = epsilon * sign_grads
    images_adv  = images - delta

    # run session
    sess = keras.backend.get_session()
    adv_out = sess.run(images_adv, feed_dict={model.input: images})
    
    return adv_out

In [ ]:
# Choose random target classes per each instance in the test set
target = (np.argmax(test_labels, axis=1) + np.random.randint(1, num_classes, size=(test_labels.shape[0]))) % num_classes
target = keras.utils.to_categorical(target, num_classes)

# Compute perturbations using TGSM
adv_images = TGSM(model, test_images, target, epsilon=0.3)
TestAttack(model, adv_images, test_images, test_labels, target, targeted=True)

## 2. PGD Implementation


*   Perform projection step only when exceeding allowed radius
*   Different constants for overall perturbation distance and iteration step size
*   Support for targeted and untargerted
*   Clipping to allowed values (0,1)
*   Numerical stability (no zero division)
*   Batch attacks in a single call



In [ ]:
def PGD(model, images, labels, epsilon=0.1, iter_eps=0.05, iterations=10, min_x=0.0, max_x=1.0, targeted=False):
    # initialization
    x = images  # batch attacks 
    stability = 1e-11
    
    for i in range(iterations):
        print(f"iteration: {i+1}")
        if targeted:  # support for targeted and untargeted
            x = TGSM(model, x, labels, iter_eps)
        else:
            x = FGSM(model, x, labels, iter_eps)

        # calculate delta and norm
        delta = x - images
        norm = tf.norm(delta, ord='euclidean', axis=[1,2])  # norm always >= 0
        stable_norm = norm + stability  # assure numerical stability

        # reshape to divide all images by norm
        orig_delta_shape = tf.shape(delta)  # keep for reshaping back
        delta = tf.reshape(delta, shape=[orig_delta_shape[0], -1,])

        # only project where exceeds epsilon radius
        projection_mask = norm > epsilon
        dimension = tf.shape(delta)[-1]
        expanded_mask = keras.backend.repeat(projection_mask, dimension)
        expanded_mask = tf.reshape(expanded_mask, [-1, dimension])
        delta = tf.where(expanded_mask, delta / stable_norm, delta)
        delta = tf.reshape(delta, shape=orig_delta_shape)
        x += delta
        x = tf.clip_by_value(x, min_x, max_x)  # clip to allowed range

        sess = keras.backend.get_session()
        x = sess.run(x)
    
    return x

## 3. Test PGD implementation over MNIST

In [ ]:
experiments = [
  {
    'labels': test_labels, 
    'iterations': 30, 
    'epsilon': 4.0, 
    'iter_eps': 0.05, 
    'targeted': False
  },
  {
    'labels': test_labels, 
    'iterations': 30, 
    'epsilon': 3.0, 
    'iter_eps': 0.03, 
    'targeted': False
  },
  {
    'labels': test_labels,
    'iterations': 60, 
    'epsilon': 3.0, 
    'iter_eps': 0.03, 
    'targeted': False
  },
  {
    'labels': target, 
    'iterations': 30, 
    'epsilon': 4.0, 
    'iter_eps': 0.05, 
    'targeted': True
  },
  {
    'labels': target, 
    'iterations': 30, 
    'epsilon': 3.0, 
    'iter_eps': 0.03, 
    'targeted': True
  },
  {
    'labels': target, 
    'iterations': 60, 
    'epsilon': 3.0, 
    'iter_eps': 0.03, 
    'targeted': True
  }
]

In [ ]:
scores = []
for i, experiment in enumerate(experiments):
    print(f"Experiment #{i+1}")
    adv_images = PGD(model, test_images, **experiment)
    ex_score = TestAttack(model, adv_images, test_images, test_labels, target, targeted=experiment['targeted'])
    scores.append(ex_score)

In [ ]:
PGD2(model, test_images, test_labels, iterations=30, epsilon=3, iter_eps=0.03, targeted=False)

# Part II - Beyond GPD

In [ ]:
def get_frame_mask(tensor):
    """
    A utility function that returns a frame numpy array
    given two dimensions, where the frame pixels are of value 0
    and non-frame pixels are of value 1
    """
    shape = tensor.shape
    frame = np.ones((shape[1], shape[2]), dtype=tensor.dtype)
    frame[0, :] = 0
    frame[shape[1] - 1, :] = 0
    frame[:, 0] = 0
    frame[:, shape[2] - 1] = 0
    frame = np.tile(frame, (shape[0], 1, 1))
    frame = np.expand_dims(frame, axis=-1)  # return to original shape
    return frame

In [ ]:
def get_black_mask(tensor):
    """
    A utility function that returns a mask numpy array
    given a tensor, where the black pixels are of value 0
    and non-black pixels are of value 1
    """
    return (tensor != 0).astype(tensor.dtype)

In [ ]:
def PGD2(model, images, labels, epsilon=0.1, iter_eps=0.05, iterations=10, min_x=0.0, max_x=1.0, targeted=False,
         restrict_frame=False, restrict_black=False):
    # initialization
    x = images  # batch attacks 
    stability = 1e-11

    if restrict_frame:
        frame_mask = tf.constant(get_frame_mask(images))
        
    if restrict_black:
        black_mask = tf.constant(get_black_mask(images))
    
    for i in range(iterations):
        print(f"iteration: {i+1}")
        if targeted:  # support for targeted and untargeted
            x = TGSM(model, x, labels, iter_eps)
        else:
            x = FGSM(model, x, labels, iter_eps)

        # calculate delta and norm
        delta = x - images
        norm = tf.norm(delta, ord='euclidean', axis=[1,2])  # norm always >= 0
        stable_norm = norm + stability  # assure numerical stability

        # reshape to divide all images by norm
        orig_delta_shape = tf.shape(delta)  # keep for reshaping back
        delta = tf.reshape(delta, shape=[orig_delta_shape[0], -1,])

        # only project where exceeds epsilon radius
        projection_mask = norm > epsilon
        dimension = tf.shape(delta)[-1]
        expanded_mask = keras.backend.repeat(projection_mask, dimension)
        expanded_mask = tf.reshape(expanded_mask, [-1, dimension])
        delta = tf.where(expanded_mask, delta / stable_norm, delta)
        delta = tf.reshape(delta, shape=orig_delta_shape)

        # restrict certain parts of the image
        if restrict_frame:
            delta = delta * frame_mask   
        if restrict_black:
            delta = delta * black_mask
            
        x += delta
        x = tf.clip_by_value(x, min_x, max_x)  # clip to allowed range

        sess = keras.backend.get_session()
        x = sess.run(x)
        
        TestAttack(model, x, images, labels, sanity_target, targeted=True)
    
    return x

In [ ]:
frame_experiments = [
  {
    'labels': test_labels, 
    'iterations': 30, 
    'epsilon': 4.0, 
    'iter_eps': 0.05, 
    'targeted': False,
    'restricted_frame': True,
    'restricted_black': False
  },
  {
    'labels': test_labels, 
    'iterations': 30, 
    'epsilon': 3.0, 
    'iter_eps': 0.03, 
    'targeted': False,
    'restricted_frame': True,
    'restricted_black': False
  },
  {
    'labels': test_labels,
    'iterations': 60, 
    'epsilon': 3.0, 
    'iter_eps': 0.03, 
    'targeted': False,
    'restricted_frame': True,
    'restricted_black': False
  },
  {
    'labels': target, 
    'iterations': 30, 
    'epsilon': 4.0, 
    'iter_eps': 0.05, 
    'targeted': True,
    'restricted_frame': True,
    'restricted_black': False
  },
  {
    'labels': target, 
    'iterations': 30, 
    'epsilon': 3.0, 
    'iter_eps': 0.03, 
    'targeted': True,
    'restricted_frame': True,
    'restricted_black': False
  },
  {
    'labels': target, 
    'iterations': 60, 
    'epsilon': 3.0, 
    'iter_eps': 0.03, 
    'targeted': True,
    'restricted_frame': True,
    'restricted_black': False
  }
]

In [ ]:
black_experiments = [
  {
    'labels': test_labels, 
    'iterations': 30, 
    'epsilon': 4.0, 
    'iter_eps': 0.05, 
    'targeted': False,
    'restricted_frame': False,
    'restricted_black': True
  },
  {
    'labels': test_labels, 
    'iterations': 30, 
    'epsilon': 3.0, 
    'iter_eps': 0.03, 
    'targeted': False,
    'restricted_frame': False,
    'restricted_black': True
  },
  {
    'labels': test_labels,
    'iterations': 60, 
    'epsilon': 3.0, 
    'iter_eps': 0.03, 
    'targeted': False,
    'restricted_frame': False,
    'restricted_black': True
  },
  {
    'labels': target, 
    'iterations': 30, 
    'epsilon': 4.0, 
    'iter_eps': 0.05, 
    'targeted': True,
    'restricted_frame': False,
    'restricted_black': True
  },
  {
    'labels': target, 
    'iterations': 30, 
    'epsilon': 3.0, 
    'iter_eps': 0.03, 
    'targeted': True,
    'restricted_frame': False,
    'restricted_black': True
  },
  {
    'labels': target, 
    'iterations': 60, 
    'epsilon': 3.0, 
    'iter_eps': 0.03, 
    'targeted': True,
    'restricted_frame': False,
    'restricted_black': True
  }
]

In [ ]:
frame_scores = []
for i, frame_experiment in enumerate(experiments):
    print(f"Experiment #{i+1}")
    adv_images = PGD(model, test_images, **experiment)
    ex_score = TestAttack(model, adv_images, test_images, test_labels, target, targeted=experiment['targeted'])
    frame_scores.append(ex_score)

In [ ]:
black_scores = []
for i, black_experiment in enumerate(experiments):
    print(f"Experiment #{i+1}")
    adv_images = PGD(model, test_images, **experiment)
    ex_score = TestAttack(model, adv_images, test_images, test_labels, target, targeted=experiment['targeted'])
    black_scores.append(ex_score)